In [1]:
import json

In [2]:
f1 = json.load(open('/raid/infolab/bhavyakohli/mmdet/mmdetection/a_tooth/train_quadrant.json', 'r'))
f2 = json.load(open('/raid/infolab/bhavyakohli/mmdet/mmdetection/a_tooth/train_quadrant_enumeration.json', 'r'))

In [3]:
f1.keys(), f2.keys()

(dict_keys(['images', 'annotations', 'categories']),
 dict_keys(['images', 'annotations', 'categories_1', 'categories_2']))

In [186]:
from tqdm import tqdm

quadrant_centers = {images['file_name']: dict() for images in f1['images']}

for i in range(len(f1['annotations'])):
    image_name = f1['images'][i//4]['file_name']
    annot = f1['annotations'][i]
    cat = annot['category_id']

    if cat == 1:
        cat_ = 0
    elif cat == 0:
        cat_ = 1
    else:
        cat_ = cat
    
    quadrant_centers[image_name][cat_] = annot['bbox'][:2]

In [190]:
for k in quadrant_centers.keys():
    centers = quadrant_centers[k]

    assert centers[0][0] < centers[1][0]

In [218]:
from copy import deepcopy
f2_new = deepcopy(f2)

In [219]:
annots_to_be_deleted = []
images_to_be_deleted = []
issues = []
for i in range(len(f2['annotations'])):
    annot, new_annot = f2['annotations'][i], f2_new['annotations'][i]
    image_id = annot['image_id'] - 1
    file_name = f2['images'][image_id]['file_name']
    try:
        centers = quadrant_centers[file_name]
    except KeyError:
        images_to_be_deleted.append(image_id)
        annots_to_be_deleted.append(i)
        continue

    x, y, h, w = annot['bbox']
    xx, yy = centers[annot['category_id_1']]
    nx, ny = [x - xx, y - yy]

    if nx < 0 or ny < 0:
        issues.append(file_name)

    new_annot['bbox'] = [max(nx,0), max(ny,0), h, w]
    
    new_annot['category_id'] = new_annot['category_id_2']

    del new_annot['category_id_1']
    del new_annot['category_id_2']

In [173]:
for i in range(4):
    print(f1['annotations'][i]) 

{'iscrowd': 0, 'image_id': 1, 'bbox': [1453.9245283018868, 332.2452830188679, 624.5283018867922, 467.92452830188677], 'segmentation': [[1474, 800, 1453, 473, 1931, 332, 2078, 539, 1789, 728]], 'category_id': 0, 'id': 1, 'area': 180569}
{'iscrowd': 0, 'image_id': 1, 'bbox': [918.0754716981131, 356.7735849056603, 556.6037735849058, 450.9433962264152], 'segmentation': [[1452, 458, 1474, 807, 1263, 764, 918, 592, 1036, 356]], 'category_id': 1, 'id': 2, 'area': 160363}
{'iscrowd': 0, 'image_id': 1, 'bbox': [1506.754716981132, 505.8301886792452, 733.9622641509434, 632.0754716981131], 'segmentation': [[1506, 805, 1533, 1137, 2240, 803, 2110, 505, 1742, 743]], 'category_id': 2, 'id': 3, 'area': 214834}
{'iscrowd': 0, 'image_id': 1, 'bbox': [733.1698113207547, 541.6792452830189, 792.4528301886792, 611.3207547169811], 'segmentation': [[1510, 796, 1525, 1153, 1144, 1081, 733, 809, 910, 541, 1287, 756]], 'category_id': 3, 'id': 4, 'area': 262232}


In [213]:
x

1441.090909090909

In [211]:
centers

{0: [596.9811320754717, 13.660377358490564],
 1: [1525.2830188679245, 11.773584905660357],
 2: [1500.754716981132, 404.2264150943396],
 3: [625.2830188679245, 402.33962264150944]}

In [215]:
for i in range(200):
    if f2['annotations'][i]['image_id'] == 3:    
        if f2['annotations'][i]['category_id_1'] == 2:
            print(f2['annotations'][i]) 

{'iscrowd': 0, 'image_id': 3, 'bbox': [1444.7272727272725, 667.5454545454545, 70.90909090909099, 281.81818181818176], 'segmentation': [[1453, 671, 1501, 667, 1506, 732, 1515, 949, 1444, 943, 1446, 703]], 'id': 69, 'area': 17517, 'category_id_1': 2, 'category_id_2': 0}
{'iscrowd': 0, 'image_id': 3, 'bbox': [1499.2727272727273, 658.4545454545454, 83.63636363636351, 310.9090909090909], 'segmentation': [[1504, 740, 1499, 660, 1582, 658, 1564, 752, 1566, 967, 1522, 969]], 'id': 70, 'area': 17742, 'category_id_1': 2, 'category_id_2': 1}
{'iscrowd': 0, 'image_id': 3, 'bbox': [1553.8181818181818, 662.090909090909, 110.90909090909076, 361.81818181818176], 'segmentation': [[1571, 789, 1553, 676, 1577, 662, 1631, 669, 1646, 718, 1664, 1003, 1617, 1023]], 'id': 71, 'area': 24583, 'category_id_1': 2, 'category_id_2': 2}
{'iscrowd': 0, 'image_id': 3, 'bbox': [1639.2727272727273, 652.9999999999999, 107.27272727272725, 347.27272727272737], 'segmentation': [[1666, 783, 1644, 725, 1639, 698, 1668, 652, 

In [183]:
for i in range(27,50,1):
    if f2['annotations'][i]['category_id_1'] == 1:
        print(f2['annotations'][i]['bbox']) 

[1386.1891891891892, 400.7297297297297, 70.2702702702702, 286.48648648648657]
[1432.1351351351352, 381.81081081081084, 86.48648648648646, 278.37837837837833]
[1499.7027027027027, 365.5945945945946, 78.37837837837833, 305.4054054054054]
[1572.6756756756756, 362.8918918918919, 86.48648648648646, 272.972972972973]
[1621.3243243243244, 365.5945945945946, 124.32432432432438, 270.2702702702703]
[1705.1081081081081, 330.4594594594595, 148.64864864864876, 262.1621621621622]
[1791.5945945945946, 271.0, 200.0, 264.8648648648649]


In [ ]:
"height": 1316, "width": 2788

In [180]:
quadrant_centers['train_435.png']

{1: [459.25, 235.25],
 0: [958.0, 216.5],
 2: [968.0, 420.25],
 3: [411.75, 435.25]}

In [161]:
quadrant_centers['train_436.png']

{0: [1466.7272727272727, 232.96969696969688],
 1: [772.7878787878788, 287.5151515151515],
 2: [1472.7878787878788, 502.6666666666665],
 3: [612.1818181818181, 499.6363636363635]}

In [220]:
for i in range(len(annots_to_be_deleted)):
    tbd = annots_to_be_deleted[i]
    del f2_new['annotations'][tbd]
    annots_to_be_deleted = [j-1 for j in annots_to_be_deleted]

for itbd in set(images_to_be_deleted):
    del f2_new['images'][itbd]

In [154]:
f2_new['categories'] = f2_new['categories_2']

del f2_new['categories_1']
del f2_new['categories_2']

In [155]:
for i in f2_new['images']:
    assert i['file_name'] in quadrant_centers.keys()

In [156]:
json.dump(f2_new, open('/raid/infolab/bhavyakohli/mmdet/mmdetection/a_tooth/train_enumeration.json', 'w'))

In [105]:
f2['images'][2]

{'height': 1316, 'width': 2888, 'id': 3, 'file_name': 'train_95.png'}

In [102]:
f2['annotations'][60]

{'iscrowd': 0,
 'image_id': 3,
 'bbox': [1037.4545454545453,
  294.81818181818176,
  107.27272727272725,
  321.81818181818176],
 'segmentation': [[1037,
   354,
   1037,
   427,
   1048,
   500,
   1053,
   534,
   1046,
   567,
   1046,
   592,
   1086,
   616,
   1124,
   600,
   1144,
   572,
   1130,
   518,
   1124,
   485,
   1086,
   294,
   1042,
   305]],
 'id': 61,
 'area': 22550,
 'category_id_1': 0,
 'category_id_2': 4}